In [ ]:
import os,sys,inspect
from IPython.display import display, Markdown, Latex, Javascript, clear_output
import emoji
import time
import ipywidgets as widgets
import ipython_blocking
import numpy as np
import time
from tqdm import tqdm_notebook
import functools
import configparser
import webbrowser
import shutil
import json

parentdir = os.path.abspath('../../../pymodules')

sys.path.insert(0,parentdir)
#from connect_device import connect
from connect_device_v2.vizualizate import vizulizated_finds_devices, advanced_search_device,DeviceBundle
from connect_device_v2.save_file import SaveFilesButton
from connect_device_v2.connect import connect_to_device
from connect_device_v2.find_device import FindDevice
from connect_device_v2.auxiliary_commands import check_error,disable_all_correctors,enable_ifbf_for_all_segment
import pandas as pd
from datetime import datetime
button = widgets.Button(
                description='Continue',
                disabled=False,
                button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
                tooltip='Click me',
                icon='play'        
                )
testname="Тест СКО"
display(Markdown('''### ░░░▄▀▀▀▄░ГУСЬ░

▄███▀░◐░░░▌░░░░░░░

░░░░▌░░░░░▐░░░░░░░

░░░░▐░░░░░▐░░░░░░░

░░░░▌░░░░░▐▄▄░░░░░

░░░░▌░░░░▄▀▒▒▀▀▀▀▄

░░░▐░░░░▐▒▒▒▒▒▒▒▒▀▀▄

░░░▐░░░░▐▄▒▒▒▒▒▒▒▒▒▒▀▄

░░░░▀▄░░░░▀▄▒▒▒▒▒▒▒▒▒▒▀▄

░░░░░░▀▄▄▄▄▄█▄▄▄▄▄▄▄▄▄▄▄▀▄

░░░░░░░░░░░▌▌░▌▌░░░░░

░░░░░░░░░░░▌▌░▌▌░░░░░

░░░░░░░░░▄▄▌▌▄▌▌░░░░░ '''))
display(f'{testname}')

In [ ]:
from connect_device_v2.find_device import FindDevice
box = FindDevice()
display(box)


In [ ]:
button = box.select_device_continue_button

In [ ]:
get_ipython().run_line_magic('blockrun', 'button')

In [ ]:
button1 = box.select_config_continue_button

In [ ]:
get_ipython().run_line_magic('blockrun', 'button1')

In [ ]:
inst = box.selected_device

In [ ]:
inst.write("SYST:PRES")

In [ ]:
# date=datetime.now()
# date=date.strftime('%Y-%m-%d %H:%M:%S')
# device = inst.query("*IDN?")
# device = device.replace(' ','')
# device = device.split(',')
# device_info = {'Время запуска':date,
#               'device':device[0],
#               'model':device[1],
#               'serial':device[2],
#               'version':device[3]}
# #inst2 = ng.instance()
# #путь к файлу с сегментами
# currentDir = get_ipython().run_line_magic('pwd', '')
# #resultPath= currentDir +'\\frequency_plan.seg'
# portCount=int(inst.query("SERV:PORT:COUN?"))
# traces=["S","R","T"]
# traces_count = portCount* len(traces)

In [ ]:
result_config =  box.config

In [ ]:
def HZ_convert(pref, hertz):
    try:
        hertz = int(hertz)
    except ValueError:
        return -1
    if pref == "k":
        return hertz * 1000
    if pref == "M":
        return hertz * 1000000
    if pref == "G":
        return hertz * 1000000000
    return -1


# функция однократного россчерка
def one_scan(ch_num):
    inst.write('TRIG:SOUR BUS')  # устанавливаем источник триггера на шину
    inst.write(f"INIT{ch_num}:CONT OFF")  # включаем повторный режим запуска триггера
    # переходим в состояние ожидания триггера
    inst.write('INIT')
    inst.write('TRIG:SING')  # выполняем россчерк
    inst.query('*OPC?')

In [ ]:
def sko_algorithm(trace_count, bwid, att):
    math_stat = []

    # Устанавливаем фильтр ПЧ 300 или 3 кГц
    inst.write(f'SENS1:BWID {HZ_convert("k", f"{bwid}")}')

    # вкл. Аттенюатор 10, 30 или 50 дБ
    inst.write('SERVice:RFCTL:POWer:STATe 1')
    inst.write(f'SERVice:RFCTL:POWer:ATT {att}')

    # Частотный план
    inst.write('SERVice:SWEep:FREQuency:FACTory')

    # Код ЦАП.  АЧХ
#     write(CMT, 'SERV:RFCTL:POW:DAC 6554')

    # Создание трасс и росчерк
    inst.write(f'CALC1:PAR:COUN 1')
    for i in range(trace_count*3):
        params = ['S', 'T', 'R']
        # T, R
        if i%3 != 0:
            inst.write(f'CALC1:PAR1:DEF {params[i % 3]}{(i // 3) + 1}')
            # установка порта-источника
            inst.write(f'CALC1:PAR1:SPOR {(i//3) + 1}')
        # S
        else:
            inst.write(f'CALC1:PAR1:DEF {params[i%3]}{(i // 3) + 1}{(i // 3) + 1}')
        one_scan(1)

        # Сохранение текущего измерения в памяти и Data/mem
        inst.write(f'CALC1:PAR1:SEL')
        inst.write(f'CALC1:MATH:MEM')
        inst.write(f'CALC1:MATH:FUNC DIV')
        one_scan(1)

        # запись СКО
        inst.write(f'CALC1:MST ON')
        math_stat.append(inst.query(f'CALC1:MST:DATA?').split(','))
    return math_stat

In [ ]:
def math_stat_foo(mass, iff):
    trace_num = int(inst.query("SERV:PORT:COUN?"))
    math_stats = []
    # mass = [10,30,50, 100]
    # iff = [300,3, 2]
    for i in mass:      # Алгоритм для ПЧ 300 и 3 кГц, аттенюатора 10, 30, 50 дБ
        for j in iff:
            math_stat = sko_algorithm(trace_num, j, i)
            math_stats.append(math_stat)


    math_result = []    # Для совмещения массивов S11 + S22 + S33, T11 + T22.....
    if trace_num != 1:
        for j in range(len(mass)*len(iff)):
            math1 = [] # S
            math2 = [] # T
            math3 = [] # R
            for i in range(trace_num):
                math1 += math_stats[j][i*3]
                math2 += math_stats[j][(i*3) + 1]
                math3 += math_stats[j][(i*3) + 2]
            arr = [math1, math2, math3]
            math_result.append(arr)
    else:
        math_result = math_stats
    return math_result

In [ ]:
def list_to_dict(arr, ports, param):    # Перевод arr в dict
    dict = {'max_value':0}
    params = []
    for i in range(ports):
        params.append(arr[1+(i*3)])
        dict.update({f'{param}{i+1}{i+1}': params[i]})
    dict['max_value'] = max(params)
    return dict

In [ ]:
import json
def masth2json(mass, iff, trace_num, math_result, json=json):
    result = {"atthenuator":{}}
    count = 0 # от 0 до 5
    for i in mass:
        result["atthenuator"][f'{i}'] = {'IF':{}}
        for j in iff:
            value_otn = list_to_dict(math_result[count][0], trace_num, 'S')
            value_t = list_to_dict(math_result[count][1], trace_num, 'T')
            value_r = list_to_dict(math_result[count][2], trace_num, 'R')
            count += 1
            value_abs = value_r | value_t
            result["atthenuator"][f'{i}']['IF'][f'{j}']={'trace':{"absolute":{},"otnosit":{}}}
            result["atthenuator"][f'{i}']["IF"][f'{j}']["trace"]["absolute"].update(value_abs)
            result["atthenuator"][f'{i}']["IF"][f'{j}']["trace"]["otnosit"].update(value_otn)

    json_data=json.dumps(result, indent=4)
    print(json_data)
    json = open('sko_result.json', 'w')
    json.write(json_data)
    json.close()

In [ ]:
def get_iff(iff_list):
    iff = []
    arr = iff_list.split(',')
    for i in range(len(arr)):
        arr1 = arr[i].split(' ')
        num = arr1[0]
        pref = arr1[1]
        iff.append(HZ_convert(pref, num))
    return iff

In [ ]:
def math2json(mass, iff, trace_num, math_result, json=json):
    result = {"atthenuator":{}}
    count = 0 # от 0 до 5
    for i in mass:
        result["atthenuator"][f'{i}'] = {'IF':{}}
        for j in iff:
            value_otn = list_to_dict(math_result[count][0], trace_num, 'S')
            value_t = list_to_dict(math_result[count][1], trace_num, 'T')
            value_r = list_to_dict(math_result[count][2], trace_num, 'R')
            count += 1
            value_abs = {**value_r, **value_t}
            result["atthenuator"][f'{i}']['IF'][f'{j}']={'trace':{"absolute":{},"otnosit":{}}}
            result["atthenuator"][f'{i}']["IF"][f'{j}']["trace"]["absolute"].update(value_abs)
            result["atthenuator"][f'{i}']["IF"][f'{j}']["trace"]["otnosit"].update(value_otn)

    json_data=json.dumps(result, indent=4)
    
    json = open('sko_result.json', 'w')
    json.write(json_data)
    json.close()
    return json_data

In [ ]:
config = box.config
iff = get_iff(config['DEVICE']['filter_IF'])
mass = (config['DEVICE']['athenuator']).split(',')
trace_num = int(inst.query('SERV:PORT:COUN?'))
math_result = math_stat_foo(mass, iff)

In [ ]:
json_data=math2json(mass, iff, trace_num, math_result, json=json)

In [ ]:
def html_table(json_data, acces):
    data = {}

    with open('sko_result.json', 'r', encoding='utf-8') as jr:
        data.update(json.load(jr))

#     access_abs = input('Введите допуск для абсолютных трасс')
#     access_otn = input('Введите допуск для относительных трасс')

#     sko_result = open('sko_result.html', 'w')
    htmlstr = '''<html>
        <style>
        table, th, td {
          border:1px solid black;
        }
        </style>
        <body>
            <table style="width:100%; margin-left:auto;margin-right:auto;">
                <tr>
                    <th>
                        Канал
                    </th>
                    <th>
                        ПЧ, Гц
                    </th>
                    <th>
                        Аттюнеатор, дБ
                    </th>
                    <th>
                        Измерение, дБ
                    </th>
                    <th>
                        Максимум, дБ
                    </th>
                    <th>
                        Допуск, дБ
                    </th>
                </tr>
            '''

    for i in data["atthenuator"]:
        for j in data["atthenuator"][i]["IF"]:   # 300/3
            for k in data["atthenuator"][i]["IF"][j]["trace"]:  # abs/otn
                count = 0
                channel_arr = []
                channel_arr_sko = []
                for g in data["atthenuator"][i]["IF"][j]["trace"][k]:   # T, R, S, max_value
                    max_value = format(float(data["atthenuator"][i]["IF"][j]["trace"][k]["max_value"]), '.8f')
                    if count != 0:  # т.к. 0 индекс max value
                        channel_arr_sko.append(format(float(data["atthenuator"][i]["IF"][j]["trace"][k][g]), '.8f'))  # запись СКО
                        channel_arr.append(g)
                    count += 1
                    if count == (len(data["atthenuator"][i]["IF"][j]["trace"][k])):     # если достали все данные одного прохода
                        # print(data["atthenuator"][i]["IF"][j]["trace"][k])
                        # print(len(data["atthenuator"][i]["IF"][j]["trace"][k]))
#                         acces = 0
#                         if k == "otnosit":
#                             acces = access_otn
#                         else:
#                             acces = access_abs
                        trace_num = len(channel_arr)

                        if float(channel_arr_sko[0]) > float(acces):    # запись строчек
                            htmlstr += f'''<tr><td>{channel_arr[0]}</td><td rowspan = {trace_num}>{j} </td><td rowspan = {trace_num}>{i}</td><td style="background-color:#FFFF00">{channel_arr_sko[0]}</td><td rowspan = {trace_num}>{max_value}</td><td rowspan = {trace_num}>{acces}</td></tr>'''
                        else:
                            htmlstr += f'''<tr><td >{channel_arr[0]}</td><td rowspan = {trace_num}>{j}</td><td rowspan = {trace_num}>{i}</td><td>{channel_arr_sko[0]}</td><td rowspan = {trace_num}>{max_value}</td><td rowspan = {trace_num}>{acces}</td></tr>'''
                        for traces in range(1, trace_num):
                            if float(channel_arr_sko[traces]) > float(acces):  # запись строчечк
                                htmlstr += f'''<tr><td>{channel_arr[traces]}</td><td style="background-color:#FFFF00">{channel_arr_sko[traces]}</td></tr>'''
                            else:
                                htmlstr += f'''<tr><td>{channel_arr[traces]}</td><td>{channel_arr_sko[traces]}</td></tr>'''

                        # if float(max_value) > float(acces):    # запись строчечк
                        #     htmlstr += f'''<tr><td>{channel_arr}</td><td>{j}</td><td>{i}</td><td style="background-color:#FFFF00">{channel_arr_sko}</td><td>{max_value}</td><td>{acces}</td></tr>'''
                        # else:
                        #     htmlstr += f'''<tr><td>{channel_arr}</td><td>{j}</td><td>{i}</td><td>{channel_arr_sko}</td><td>{max_value}</td><td>{acces}</td></tr>'''

    htmlstr += '''</table>
        </body>
    </html>'''
    return htmlstr

In [ ]:
mark=Markdown('### 🦢Укажите допуск Дб')
text=widgets.Text(
        value='',
        placeholder='Допуск',
        description='Допуск',
        disabled=False)
button = widgets.Button(
    description='Continue',
    disabled=False,
    button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='play'        
    )
display(mark,text,button)


In [ ]:
%blockrun button

In [ ]:

from IPython.core.display import display, HTML
file_data = html_table(json_data, float(text.value))
html_table=file_data


display(HTML(html_table))
my_button = SaveFilesButton(file_data)
my_button # This will display the button in the context of Jupyter Notebook